# Preparamos los datos para nuestro script


In [3]:
import urllib, json, datetime, pycurl, requests
from datetime import date
import pandas as pd 
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup #Web scrapping


In [4]:
# Montamos la URL de donde recibiremos nuestra IP pública:
url_ip="http://ip-api.com/json/?fields=status,message,continent,continentCode,country,countryCode,region,regionName,city,district,zip,lat,lon,timezone,offset,currency,isp,org,as,asname,reverse,mobile,proxy,hosting,query"

In [6]:
# Obtenemos la fecha del día en curso:
fecha = date.today()
print(fecha)

2020-10-20


# Obtenemos la IP pública de nuestra máquina con WebScrapping


In [8]:
# Requests conecta con la web y devuelve <Response [200]> si ha conectado correctamente. Lo almacenamos en response.
response = requests.get(url_ip)
print(response)

<Response [200]>


In [9]:
# Comprobamos si la conexión devuelve el valor de correcta conexión '<Response [200]>'
respuesta=str(response).find("<Response [200]>")
